In [5]:
# Not Working

# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [38]:
%%capture
! pip install mistralai

In [7]:
import wget

url = "https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py"
wget.download(url=url)

'minsearch.py'

In [20]:
import json
with open("F:\llmZoomcamp\module1\documents.json", "rt") as file:
    docs = json.load(file)

In [27]:
# Accumulating all the docs present in all the courses

documents = []
for courses in docs:
    for doc in courses['documents']:
        doc['course'] = courses['course']
        documents.append(doc) 

In [28]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [29]:
import minsearch

In [30]:
index = minsearch.Index(
    text_fields=['question','text' ,'section'],
    keyword_fields=['course']
)

In [31]:
query = "the course has already started, can i still enroll?"

In [32]:
index.fit(documents)

In [36]:
# searching through the documents

boost = {'question': 3.0, 'section':0.5}

res = index.search(
    query=query,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [41]:
res

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [40]:
# Calling Mistral-Ai Api

from dotenv import load_dotenv
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

messages = [
    ChatMessage(role="user", content="What is the best French cheese?")
]

# No streaming
chat_response = client.chat(
    model=model,
    messages=messages,
)

print(chat_response.choices[0].message.content)

Choosing the "best" French cheese can be subjective as it depends on personal taste. France is renowned for its wide variety of cheeses, with over 400 different types. Here are a few highly regarded French cheeses across various categories:

1. **Soft Cheeses:**
   - **Brie de Meaux**: A creamy, soft cheese with a bloomy rind, often considered one of the finest cheeses in the world.
   - **Camembert de Normandie**: A soft, creamy cheese with a strong flavor, similar to Brie but typically stronger in taste.

2. **Semi-Soft Cheeses:**
   - **Morbier**: Known for its distinctive layer of ash in the middle, it has a fruity and slightly nutty flavor.
   - **Reblochon**: A nutty and fruity cheese from the Alps, often used in the dish Tartiflette.

3. **Hard Cheeses:**
   - **Comté**: A firm, nutty cheese made from unpasteurized cow's milk, often compared to Gruyère.
   - **Beaufort**: A firm, nutty, and slightly sweet cheese, similar to Comté but with its own unique character.

4. **Blue Che

In [42]:
# creating prompt

prompt = '''
        You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: {question}

        CONTEXT: {context}
'''.strip()



query = "the course has already started, can i still enroll?"

boost = {'question': 3.0, 'section':0.5}

res = index.search(
    query=query,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

context = ""

for doc in res:
    context = context + f"section:{doc['section']}\nquestion:f{doc['question']}\nanswer:f{doc['text']}\n\n"


In [47]:
formated_prompt = prompt.format(question=query, context=context)

In [48]:
print(formated_prompt)

You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: the course has already started, can i still enroll?

        CONTEXT: section:General course-related questions
question:fCourse - Can I still join the course after the start date?
answer:fYes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section:General course-related questions
question:fCourse - Can I follow the course after it finishes?
answer:fYes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section:Ge

In [49]:
# No streaming
messages = [
    ChatMessage(role="user", content=f"{formated_prompt}")
]

chat_response = client.chat(
    model=model,
    messages=messages,
)

In [50]:
# Model response

print(chat_response.choices[0].message.content)

Yes, you can still enroll in the course even if it has already started. You will still be eligible to submit the homeworks, but be aware that there are deadlines for turning in the final projects.


Since we need a presistant db for the storage of the document, which is not possible using the minsearch.py 
So we are going to user elastic search for the db.

In [53]:
%%capture
! pip install elasticsearch

In [54]:
from elasticsearch import Elasticsearch

In [55]:
es_client = Elasticsearch("http://localhost:9200")

In [56]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [57]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [58]:
from tqdm import tqdm

In [59]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [01:09<00:00, 13.63it/s]


In [60]:
query = 'I just disovered the course. Can I still join it?'

In [61]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [63]:
if es_client.indices.exists(index=index_name):

hello
